## Calcul de Coefficients Regression Lineaires

Algorithme pour calculer les coefficients d’une régression quantile et par extension d’une médiane dans un espace à plusieurs dimensions.

$X$ et $Y$ désignent des vecteurs colonnes.</br>
$\beta$ désigne un vecteur ligne</br>
$W$ une matrice diagonale.

### 1. Module `random`

Générer un ensemble de points aléatoires.

In [1]:
import random

def ensemble_aleatoire(n):
    res = [random.randint(0, 100) for i in range(n)]
    res[0] = 1000
    return res

ens = ensemble_aleatoire(10)
ens

[1000, 66, 92, 28, 51, 81, 46, 1, 36, 63]

### 2. Mediane

La médiane d’un ensemble de points $\left\{X_1, ..., X_n\right\}$ est une valeur $X_M$ telle que :

$$\sum_i \mathbf{1\!\!1}_{X_i < X_m} = \sum_i \mathbf{1\!\!1}_{X_i > X_m}$$

Autrement dit, il y a autant de valeurs inférieures que supérieures à $X_M$. On obtient cette valeur en triant les éléments par ordre croissant et en prenant celui du milieu.

In [2]:
def mediane(ensemble):
    tri = list(sorted(ensemble))
    return tri[len(tri)//2]

mediane(ens)

63

### 3. Mediane modifiée

Lorsque le nombre de points est pair, la médiane peut être n’importe quelle valeur dans un intervalle.</br>
Modifier votre fonction de façon à ce que la fonction précédente retourne le milieu de la fonction.

In [3]:
def mediane(ensemble):
    tri = list(sorted(ensemble))
    if len(tri) % 2 == 0:
        m = len(tri)//2
        return (tri[m] + tri[m-1]) / 2
    else:
        return tri[len(tri)//2]

mediane(ens)

57.0

### 4. Derivée de X

Pour un ensemble de points $E=\left\{X_1, ..., X_n\right\}$, on considère la fonction suivante :

$$f(x) = \sum_{i=1}^n \left | x - X_i\right |$$

On suppose que la médiane $X_M$ de l’ensemble $E$ n’appartient pas à $E$ : $X_M \notin E$.
    
Que vaut $f'(X_M)$ ? On acceptera le fait que la médiane est le seul point dans ce cas.

#### $$f'(X_m) = - \sum_{i=1}^n \mathbf{1\!\!1}_{X_i < X_m} + \sum_{i=1}^n \mathbf{1\!\!1}_{X_i > X_m}$$

Par définition de la médiane, $f'(X_M)=0$. En triant les éléments, on montre que la $f'(x) = 0 \Longleftrightarrow x=X_m$.

### 5. Vecteur beta

On suppose qu’on dispose d’un ensemble d’observations $\left(X_i, Y_i\right)$ avec $X_i$, $Y_i \in \mathbb{R}$. La régression linéaire consiste une relation linéaire $Y_i = a X_i + b + \epsilon_i$ qui **minimise la variance du bruit**. On pose :

$$E(a, b) = \sum_i \left(Y_i - (a X_i + b)\right)^2$$

On cherche $a$, $b$ tels que :

$$a^*, b^* = \arg \min E(a, b) = \arg \min \sum_i \left(Y_i - (a X_i + b)\right)^2$$

La fonction est dérivable et on trouve :

$$\frac{\partial E(a,b)}{\partial a} = - 2 \sum_i X_i ( Y_i - (a X_i + b)) \text{ et } \frac{\partial E(a,b)}{\partial b} = - 2 \sum_i ( Y_i - (a X_i + b))$$

Il suffit alors d’annuler les dérivées. On résoud un système d’équations linéaires. On note :

$$\begin{array}{l} \mathbb{E} X = \frac{1}{n}\sum_{i=1}^n X_i \text{ et } \mathbb{E} Y = \frac{1}{n}\sum_{i=1}^n Y_i \\ \mathbb{E}{X^2} = \frac{1}{n}\sum_{i=1}^n X_i^2 \text{ et } \mathbb{E} {XY} = \frac{1}{n}\sum_{i=1}^n X_i Y_i \end{array}$$

Finalement :

$$\begin{array}{l} a^* = \frac{ \mathbb{E} {XY} - \mathbb{E} X \mathbb{E} Y}{\mathbb{E}{X^2} - (\mathbb{E} X)^2} \text{ et } b^* = \mathbb{E} Y - a^* \mathbb{E} X \end{array}$$

Lorsqu’on a plusieurs dimensions pour $X$, on écrit le problème d’optimisation, on cherche les coefficients $\beta^*$ qui minimisent :

$$E(\beta)=\sum_{i=1}^n \left(y_i - X_i \beta\right)^2 = \left \Vert Y - X\beta \right \Vert ^2$$

La solution est : 

$$\beta^* = (X'X)^{-1}X'Y$$.

Ecrire une fonction qui calcule ce vecteur optimal.

In [4]:
from numpy.linalg import inv

def regression_lineaire(X, Y):
    t = X.T
    return inv(t @ X) @ t @ Y

import numpy
X = numpy.array(ens).reshape((len(ens), 1))

# test pour check si la valeur n'est pas aberrante
regression_lineaire(X, X+1)   

array([[1.00142116]])

### 6. Matrice diagonale

Ecrire une fonction qui transforme un vecteur en une matrice diagonale.

In [5]:
def matrice_diagonale(W):
    return numpy.diag(W)

matrice_diagonale([1, 2, 3])

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

### 7. Matrice de regression pondérée

On considère maintenant que chaque observation est pondérée par un poids $w_i$. On veut maintenant trouver le vecteur $\beta$ qui minimise :

$$E(\beta)=\sum_{i=1}^n w_i \left( y_i - X_i \beta \right)^2 = \left \Vert W^{\frac{1}{2}}(Y - X\beta)\right \Vert^2$$

Où $W=diag(w_1, ..., w_n)$ est la matrice diagonale. La solution est :

$$\beta_* = (X'WX)^{-1}X'WY$$

Ecrire une fonction qui calcule la solution de la régression pondérée (La fonction ravel est utile).

In [6]:
def regression_lineaire_ponderee(X, Y, W):
    if len(W.shape) == 1 or W.shape[0] != W.shape[1]:
        # c'est un vecteur
        W = matrice_diagonale(W.ravel())
        
    wx = W @ X
    xt = X.T
    
    return inv(xt @ wx) @ xt @ W @ Y

In [7]:
X = numpy.array(sorted(ens)).reshape((len(ens), 1))
Y = X.copy()
Y[0] = max(X)
W = numpy.ones(len(ens))
W[0] = 0
regression_lineaire_ponderee(X, Y, W), regression_lineaire(X, Y)

(array([[1.]]), array([[1.00096976]]))

### 8. Fonctions max, reciproques

Ecrire une fonction qui calcule les quantités suivantes (fonctions [reciproque](https://numpy.org/doc/stable/reference/generated/numpy.reciprocal.html), [maximum](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html))

$$z_i = \frac{1}{\max(\sigma |y_i - X_i\beta|)}$$

In [10]:
def calcule_z(X, beta, Y, W, delta=0.0001):
    epsilon = numpy.abs(Y - X @ beta)
    return numpy.reciprocal(numpy.maximum(epsilon, numpy.ones(epsilon.shape) * delta))

In [11]:
calcule_z(X * 1.0, numpy.array([[1.01]]), Y, W)

array([[1.00101102e-03],
       [3.57142857e+00],
       [2.77777778e+00],
       [2.17391304e+00],
       [1.96078431e+00],
       [1.58730159e+00],
       [1.51515152e+00],
       [1.23456790e+00],
       [1.08695652e+00],
       [1.00000000e-01]])

### 9. Codage de l'algorithme 

On souhaite coder l’algorithme suivant :

1. $w_i^{(1)} = 1$

2. $\beta_{(t)} = (X'W^{(t)}X)^{-1}X'W^{(t)}Y$

3. $w_i^{(t+1)} = \frac{1}{\max\left( \delta, \left|y_i - X_i \beta^{(t)}\right|\right)}$

4. $t = t+1$


5. Retour à l’étape 2.

In [12]:
def algorithm(X, Y, delta=0.0001):
    W = numpy.ones(X.shape[0])
    
    for i in range(0, 10):
        beta = regression_lineaire_ponderee(X, Y, W)
        W = calcule_z(X, beta, Y, W, delta=delta)
        E = numpy.abs(Y - X @ beta).sum()
        print(i, E, beta)
        
    return beta

In [13]:
X = numpy.random.rand(10, 1)
Y = X*2 + numpy.random.rand()

Y[0] = Y[0] + 100
algorithm(X, Y)

0 147.00577125582032 [[17.45956329]]
1 108.48937453480733 [[5.58302859]]
2 105.21282050915539 [[4.27360453]]
3 104.78437688161772 [[3.98931339]]
4 104.39631611249858 [[3.73181805]]
5 104.26992269551974 [[3.61895534]]
6 104.26143644862076 [[3.59924946]]
7 104.25141484984752 [[3.57597835]]
8 104.24095695453524 [[3.55169411]]
9 104.2314071731347 [[3.52951861]]


array([[3.52951861]])

### 10. Algorithme 

In [14]:
ens = ensemble_aleatoire(10)
Y = numpy.empty((len(ens), 1))
Y[:,0] = ens
X = numpy.ones((len(ens), 1))
mediane(ens)

57.0

In [15]:
Y.mean(axis=0)

array([146.8])

In [16]:
regression_lineaire(X, Y)

array([[146.8]])

In [17]:
algorithm(X,Y)

0 1706.3999999999999 [[146.8]]
1 1081.7531338731296 [[66.43828347]]
2 1074.6995224508144 [[64.34976123]]
3 1073.1101046641663 [[63.55505233]]
4 1070.791609508834 [[62.39580475]]
5 1068.4632729306418 [[61.23163647]]
6 1068.0 [[60.84711933]]
7 1068.0 [[60.84711933]]
8 1068.0 [[60.84711933]]
9 1068.0 [[60.84711933]]


array([[60.84711933]])

In [18]:
mediane(ens)

57.0

In [19]:
list(sorted(ens))

[2, 45, 47, 53, 53, 61, 65, 68, 74, 1000]

La régression linéaire égale la moyenne, l’algorithme s’approche de la médiane.

#### Quelques explications et démonstrations

Cet énoncé est inspiré de [Iteratively reweighted least squares](https://en.wikipedia.org/wiki/Iteratively_reweighted_least_squares). Cet algorithme permet notamment d’étendre la notion de médiane à des espaces vectoriels de plusieurs dimensions. On peut détermine un point $X_M$ qui minimise la quantité :

$\sum_{i=1}^n \left| X_i - X_M \right |$

Nous reprenons l’algorithme décrit ci-dessus :

1. $w_i^{(1)} = 1$

2. $\beta_{(t)} = (X'W^{(t)}X)^{-1}X'W^{(t)}Y$

3. $w_i^{(t+1)} = \frac{1}{\max\left( \delta, \left|y_i - X_i \beta^{(t)}\right|\right)}$

4. $t = t+1$

5. Retour à l’étape 2.

L’erreur quadratique pondéré est :

$E_2(\beta, W) = \sum_{i=1}^n w_i \left\Vert Y_i - X_i \beta \right\Vert^2$

$Si w_i = \frac{1}{\left|y_i - X_i \beta\right|}$, on remarque que :

$E_2(\beta, W) = \sum_{i=1}^n \frac{\left\Vert Y_i - X_i \beta \right\Vert^2}{\left|y_i - X_i \beta\right|} = \sum_{i=1}^n \left|y_i - X_i \beta\right| = E_1(\beta)$

On retombe sur l’erreur en valeur absolue optimisée par la régression quantile. Comme l’étape 2 consiste à trouver les coefficients $\beta$ qui minimise $E_2(\beta, W^{(t)})$, par construction, il ressort que :

$E_1(\beta^{(t+1)}) = E_2(\beta^{(t+1)}, W^{(t)}) \leqslant E_2(\beta^{(t)}, W^{(t)}) = E_1(\beta^{(t)})$

La suite $t \rightarrow E_1(\beta^{(t)})$ est suite décroissant est minorée par $0$. Elle converge donc vers un minimum. Or la fonction $\beta \rightarrow E_1(\beta)$ est une **fonction convexe**. Elle n’admet qu’un seul minimum (mais pas nécessaire un seul point atteignant ce minimum). L’algorithme converge donc vers la médiane. Le paramètre $\delta$ est là pour éviter les erreurs de divisions par zéros et les approximations de calcul faites par l’ordinateur.

### Quelques commentaires sur le code

Le symbol [@](https://www.python.org/dev/peps/pep-0465/) a été introduit par Python 3.5 et est équivalent à la fonction `numpy.dot`. Les dimensions des matrices posent souvent quelques problèmes.

In [20]:
import numpy
y = numpy.array([1, 2, 3])
M = numpy.array([[3, 4], [6, 7], [3, 3]])
M.shape, y.shape

((3, 2), (3,))

In [21]:
try:
    M @ y
except Exception as e:
    print(e)

matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)


Par défaut, numpy considère un vecteur de taille (3,) comme un vecteur ligne (3,1). Donc l’expression suivante va marcher :

In [22]:
y @ M

array([24, 27])

Ou :

In [23]:
M.T @ y

array([24, 27])